In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete 

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [2]:
#District Summary 

total_schools = school_data_complete["school_name"].nunique()

total_students = school_data_complete.shape[0]

unique_budget = school_data_complete["budget"].unique()
total_budget = unique_budget.sum()

avg_math = school_data_complete["math_score"].mean()

avg_reading = school_data_complete["reading_score"].mean()

math_filter = school_data_complete["math_score"] >= 70
math_passed = math_filter.value_counts()[True]
math_percentage = (math_passed / total_students) * 100 

reading_filter = school_data_complete["reading_score"] >= 70
reading_passed = reading_filter.value_counts()[True]
reading_percentage = (reading_passed / total_students) * 100 

overall_percentage = (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
both_passed = overall_percentage.value_counts()[True]
both_percentage = (both_passed/ total_students) * 100


district_summary = [[total_schools, total_students, total_budget, avg_math, avg_reading, math_percentage, reading_percentage, both_percentage]]
district_summary_df = pd.DataFrame(district_summary)
district_summary_df.columns = ["Total Schools", "Total Students", "Total Budget in Dollars", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "%Overall Passing"]
district_summary_df


,Total Schools,Total Students,Total Budget in Dollars,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [3]:
#School Summary 

#pull / build necessary info
school_data_complete["Passed Math?"] = math_filter
school_data_complete["Passed Reading?"] = reading_filter 
school_data_complete["Passed Both?"] = overall_percentage 

#School General Data 

school_type = school_data_complete["type"]

school_budget = school_data_complete["budget"]
                                              
school_data_complete["Budget Per Student"] = school_data_complete["budget"] / school_data_complete["size"]

budget_per_stu = school_data_complete["Budget Per Student"]

ind_schools_list = school_data_complete["school_name"].unique()

ind_school_type = school_data_complete.groupby(["school_name"])["type"].value_counts()

ind_school_students = school_data_complete.groupby(["school_name"])["Student ID"].count()

ind_avg_math = school_data_complete.groupby(["school_name"])["math_score"].mean()

ind_avg_reading = school_data_complete.groupby(["school_name"])["reading_score"].mean()   


reduced = school_data_complete[["school_name","type","size","budget","Budget Per Student"]]
grouped_reduced = reduced.groupby(["school_name"]).value_counts()
group_df= pd.DataFrame(grouped_reduced)


#Average Merge 
ind_merge1=pd.merge(ind_avg_math,ind_avg_reading, on="school_name")
group_df= group_df.join(ind_merge1)


#Conditional Data 

school_math_pass = school_data_complete.loc[(school_data_complete["Passed Math?"] == True), :]
school_math_pass_school = school_math_pass.groupby(["school_name"])["Passed Math?"].count()
math_passed_ind_school = pd.DataFrame(school_math_pass_school)


school_reading_pass = school_data_complete.loc[(school_data_complete["Passed Reading?"] == True), :]
school_reading_pass_school = school_reading_pass.groupby(["school_name"])["Passed Reading?"].count()
reading_passed_ind_school = pd.DataFrame(school_reading_pass_school)


school_both_pass = school_data_complete.loc[(school_data_complete["Passed Both?"] == True), :]
school_both_pass_school = school_both_pass.groupby(["school_name"])["Passed Both?"].count()
both_passed_ind_school = pd.DataFrame(school_both_pass_school)

#Merge and Clean 
passing_merge = pd.merge(math_passed_ind_school,reading_passed_ind_school,on="school_name")
passing_merge2 = pd.merge(passing_merge,both_passed_ind_school,on="school_name")
merge3= group_df.join(passing_merge2)
merge4= merge3.rename(columns={0 :"Sizes"})

merge4["Percentage Passing Math"] = (merge4["Passed Math?"] / merge4["Sizes"]) *100
merge4["Percentage Passing Reading"] = (merge4["Passed Reading?"] / merge4["Sizes"]) *100
merge4["Percentage Passing Both"] = (merge4["Passed Both?"] / merge4["Sizes"]) *100


del merge4["Sizes"]
del merge4["Passed Math?"]
del merge4["Passed Reading?"]
del merge4["Passed Both?"]
merge4


,,,,,math_score,reading_score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Both
school_name,type,size,budget,Budget Per Student,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [4]:
top_performing = merge4.sort_values("Percentage Passing Both",ascending= False )
top_performing.head(5)

,,,,,math_score,reading_score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Both
school_name,type,size,budget,Budget Per Student,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [5]:
worst_performing = merge4.sort_values("Percentage Passing Both" )
worst_performing.head(5)

,,,,,math_score,reading_score,Percentage Passing Math,Percentage Passing Reading,Percentage Passing Both
school_name,type,size,budget,Budget Per Student,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [6]:

school_reduced = school_data_complete[["school_name", "grade", "math_score"]]


ninth_grade = school_reduced.loc[(school_data_complete["grade"] == "9th"), :]
ninth_grade_avg_by_school = ninth_grade.groupby(["school_name"]).mean()


tenth_grade = school_reduced.loc [(school_data_complete["grade"] == "10th"), :] 
tenth_grade_avg_by_school = tenth_grade.groupby(["school_name"]).mean()

eleventh_grade = school_reduced.loc [(school_data_complete["grade"] == "11th"), :] 
eleventh_grade_avg_by_school = eleventh_grade.groupby(["school_name"]).mean()

twelfth_grade = school_reduced.loc [(school_data_complete["grade"] == "12th"), :] 
twelfth_grade_avg_by_school = twelfth_grade.groupby(["school_name"]).mean()

merge_1=pd.merge(ninth_grade_avg_by_school,tenth_grade_avg_by_school, on="school_name")
merge_2=pd.merge(merge_1, eleventh_grade_avg_by_school, on="school_name")
merge_3=pd.merge(merge_2, twelfth_grade_avg_by_school, on="school_name")
merge_3.columns = ["9th", "10th", "11th", "12th"]
merge_3


/var/folders/yf/1sw8j9bs3sb2hpytj0_80bvw0000gn/T/ipykernel_12524/2926539739.py:19: FutureWarning: Passing 'suffixes' which cause duplicate columns {'math_score_x'} in the result is deprecated and will raise a MergeError in a future version.
  merge_3=pd.merge(merge_2, twelfth_grade_avg_by_school, on="school_name")


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [7]:
school_reduced_r = school_data_complete[["school_name", "grade", "reading_score"]]


ninth_grade_r = school_reduced_r.loc[(school_data_complete["grade"] == "9th"), :]
ninth_grade_avg_by_school_r = ninth_grade_r.groupby(["school_name"]).mean()


tenth_grade_r = school_reduced_r.loc [(school_data_complete["grade"] == "10th"), :] 
tenth_grade_avg_by_school_r = tenth_grade_r.groupby(["school_name"]).mean()

eleventh_grade_r = school_reduced_r.loc [(school_data_complete["grade"] == "11th"), :] 
eleventh_grade_avg_by_school_r = eleventh_grade_r.groupby(["school_name"]).mean()

twelfth_grade_r = school_reduced_r.loc [(school_data_complete["grade"] == "12th"), :] 
twelfth_grade_avg_by_school_r = twelfth_grade_r.groupby(["school_name"]).mean()

merge_1_r=pd.merge(ninth_grade_avg_by_school_r,tenth_grade_avg_by_school_r, on="school_name")
merge_2_r=pd.merge(merge_1_r, eleventh_grade_avg_by_school_r, on="school_name")
merge_3_r=pd.merge(merge_2_r, twelfth_grade_avg_by_school_r, on="school_name")
merge_3_r.columns = ["9th", "10th", "11th", "12th"]
merge_3_r

/var/folders/yf/1sw8j9bs3sb2hpytj0_80bvw0000gn/T/ipykernel_12524/380771463.py:19: FutureWarning: Passing 'suffixes' which cause duplicate columns {'reading_score_x'} in the result is deprecated and will raise a MergeError in a future version.
  merge_3_r=pd.merge(merge_2_r, twelfth_grade_avg_by_school_r, on="school_name")


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [21]:
#Scores by School Spending 

spend_bins = [0,584,629,644,680]
spend_bins_names = ["<$585", "$585-$629", "$630-$644", "$645-$680"]

spend_binning = pd.cut(school_data_complete["Budget Per Student"], spend_bins, labels=spend_bins_names, include_lowest=True )

school_data_complete["Spend Bin"] = spend_binning



spend_data = school_data_complete[["Spend Bin", "reading_score", "math_score", "Passed Math?", "Passed Reading?","Passed Both?"]]

#Tier 1 General Data 
tier_1_data = spend_data.loc[(spend_data["Spend Bin"] == "<$585"), :]
tier_1_count = tier_1_data["Spend Bin"].count()
tier_1_math_avg_per = tier_1_data["math_score"].mean()
tier_1_reading_avg_per = tier_1_data["reading_score"].mean()

#Tier 2 General Data 
tier_2_data = spend_data.loc[(spend_data["Spend Bin"] == "$585-$629"), :]
tier_2_count = tier_2_data["Spend Bin"].count()
tier_2_math_avg_per = tier_2_data["math_score"].mean()
tier_2_reading_avg_per = tier_2_data["reading_score"].mean()

#Tier 3 General Data 
tier_3_data = spend_data.loc[(spend_data["Spend Bin"] == "$630-$644"), :]
tier_3_count = tier_3_data["Spend Bin"].count()
tier_3_math_avg_per = tier_3_data["math_score"].mean()
tier_3_reading_avg_per = tier_3_data["reading_score"].mean()

#Tier 4 General Data 
tier_4_data = spend_data.loc[(spend_data["Spend Bin"] == "$645-$680"), :]
tier_4_count = tier_4_data["Spend Bin"].count()
tier_4_math_avg_per = tier_4_data["math_score"].mean()
tier_4_reading_avg_per = tier_4_data["reading_score"].mean()

#Math Calculations 

spend_math_pass = spend_data.loc[(spend_data["Passed Math?"] == True), :]

tier_1_math_pass = spend_math_pass.loc[(spend_data["Spend Bin"] == "<$585"), :]
tier_1_math_pass_count = tier_1_math_pass["Spend Bin"].count()

tier_2_math_pass = spend_math_pass.loc[(spend_data["Spend Bin"] == "$585-$629"), :]
tier_2_math_pass_count = tier_2_math_pass["Spend Bin"].count()

tier_3_math_pass = spend_math_pass.loc[(spend_data["Spend Bin"] == "$630-$644"), :]
tier_3_math_pass_count = tier_3_math_pass["Spend Bin"].count()

tier_4_math_pass = spend_math_pass.loc[(spend_data["Spend Bin"] == "$645-$680"), :]
tier_4_math_pass_count = tier_4_math_pass["Spend Bin"].count()

#Math Percentages

tier_1_math_per = (tier_1_math_pass_count/tier_1_count) *100
tier_2_math_per = (tier_2_math_pass_count/tier_2_count) *100
tier_3_math_per = (tier_3_math_pass_count/tier_3_count) *100
tier_4_math_per = (tier_4_math_pass_count/tier_4_count) *100

#Reading Calculations 

spend_reading_pass = spend_data.loc[(spend_data["Passed Reading?"] == True), :]

tier_1_reading_pass = spend_reading_pass.loc[(spend_data["Spend Bin"] == "<$585"), :]
tier_1_reading_pass_count = tier_1_reading_pass["Spend Bin"].count()

tier_2_reading_pass = spend_reading_pass.loc[(spend_data["Spend Bin"] == "$585-$629"), :]
tier_2_reading_pass_count = tier_2_reading_pass["Spend Bin"].count()

tier_3_reading_pass = spend_reading_pass.loc[(spend_data["Spend Bin"] == "$630-$644"), :]
tier_3_reading_pass_count = tier_3_reading_pass["Spend Bin"].count()

tier_4_reading_pass = spend_reading_pass.loc[(spend_data["Spend Bin"] == "$645-$680"), :]
tier_4_reading_pass_count = tier_4_reading_pass["Spend Bin"].count()

#Reading Percentages

tier_1_reading_per = (tier_1_reading_pass_count/tier_1_count) *100
tier_2_reading_per = (tier_2_reading_pass_count/tier_2_count) *100
tier_3_reading_per = (tier_3_reading_pass_count/tier_3_count) *100
tier_4_reading_per = (tier_4_reading_pass_count/tier_4_count) *100

#Both Passing of Each Size

spend_both_pass = spend_data.loc[(spend_data["Passed Both?"] == True), :]

tier_1_both_pass = spend_both_pass.loc[(spend_data["Spend Bin"] == "<$585"), :]
tier_1_both_pass_count = tier_1_both_pass["Spend Bin"].count()

tier_2_both_pass = spend_both_pass.loc[(spend_data["Spend Bin"] == "$585-$629"), :]
tier_2_both_pass_count = tier_2_both_pass["Spend Bin"].count()

tier_3_both_pass = spend_both_pass.loc[(spend_data["Spend Bin"] == "$630-$644"), :]
tier_3_both_pass_count = tier_3_both_pass["Spend Bin"].count()

tier_4_both_pass = spend_both_pass.loc[(spend_data["Spend Bin"] == "$645-$680"), :]
tier_4_both_pass_count = tier_4_both_pass["Spend Bin"].count()

#Reading Passing percentage  of Each Size 

tier_1_both_per = (tier_1_both_pass_count/tier_1_count) *100
tier_2_both_per = (tier_2_both_pass_count/tier_2_count) *100
tier_3_both_per = (tier_3_both_pass_count/tier_3_count) *100
tier_4_both_per = (tier_4_both_pass_count/tier_4_count) *100

#Build DF

spend_summary_df = pd.DataFrame({"Spending Ranges Per Student": ["<$585", "$585-$629", "$630-$644", "$645-$680"],
                                       "Average Math Score": [tier_1_math_avg_per, tier_2_math_avg_per, tier_3_math_avg_per,tier_4_math_avg_per],
                                       "Average Reading Score":[tier_1_reading_avg_per, tier_2_reading_avg_per, tier_3_reading_avg_per,tier_4_reading_avg_per],
                                       "% Passing Math": [tier_1_math_per, tier_2_math_per, tier_3_math_per,tier_4_math_per],
                                       "% Passing Reading": [tier_1_reading_per, tier_2_reading_per, tier_3_reading_per,tier_4_reading_per],
                                       "%Overall Passing": [tier_1_both_per, tier_2_both_per, tier_3_both_per, tier_4_both_per]})
                                       
                                       

spend_summary_df


,Spending Ranges Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
0,<$585,83.363065,83.964039,93.702889,96.686558,90.640704
1,$585-$629,79.982873,82.312643,79.109851,88.513145,70.939239
2,$630-$644,77.821056,81.301007,70.623565,82.600247,58.841194
3,$645-$680,77.049297,81.005604,66.230813,81.109397,53.528791


In [25]:
#Scores by School Size 

#convert size to bins 

size_bins = [0,999,1999,4999]
size_bins_names = ["Small (<1000)", "Medium(1000-2000)", "Large (2000-5000)"]

bin_df = pd.cut(school_data_complete["size"], size_bins, labels=size_bins_names, include_lowest=True )

#pull/organize necessary info 
school_data_complete["Bin"] = bin_df

size_columns = school_data_complete[["Bin", "reading_score", "math_score", "Passed Math?", "Passed Reading?","Passed Both?"]]


#Population of each Size 
small_return = size_columns.loc[(size_columns["Bin"] == "Small (<1000)"), :]
small_count = small_return["Bin"].count()

medium_return = size_columns.loc[(size_columns["Bin"] == "Medium(1000-2000)"), :]
medium_count = medium_return["Bin"].count()

large_return = size_columns.loc[(size_columns["Bin"] == "Large (2000-5000)"), :]
large_count = large_return["Bin"].count()


# Average Scores by School Size Calculations 

small_score_avg = size_columns.loc[(size_columns["Bin"] == "Small (<1000)"), :]
small_math_avg_per = small_score_avg["math_score"].mean()
small_reading_avg_per = small_score_avg["reading_score"].mean()


medium_score_avg = size_columns.loc[(size_columns["Bin"] == "Medium(1000-2000)"), :]
medium_math_avg_per = medium_score_avg["math_score"].mean()
medium_reading_avg_per = medium_score_avg["reading_score"].mean()

large_score_avg = size_columns.loc[(size_columns["Bin"] == "Large (2000-5000)"), :]
large_math_avg_per = large_score_avg["math_score"].mean()
large_reading_avg_per = large_score_avg["reading_score"].mean()



# Math Passing of Each Size

size_math_pass = size_columns.loc[(size_columns["Passed Math?"] == True), :]

small_math_pass = size_math_pass.loc[(size_columns["Bin"] == "Small (<1000)"), :]
small_math_pass_count = small_math_pass["Bin"].count()

medium_math_pass = size_math_pass.loc[(size_columns["Bin"] == "Medium(1000-2000)"), :]
medium_math_pass_count = medium_math_pass["Bin"].count()

large_math_pass = size_math_pass.loc[(size_columns["Bin"] == "Large (2000-5000)"), :]
large_math_pass_count = large_math_pass["Bin"].count()


#Math Passing percentae  of Each Size 

small_math_per = (small_math_pass_count/small_count) *100
medium_math_per = (medium_math_pass_count/medium_count) *100
large_math_per = (large_math_pass_count/large_count) *100

#Reading Passing of Each Size

size_reading_pass = size_columns.loc[(size_columns["Passed Reading?"] == True), :]

small_reading_pass = size_reading_pass.loc[(size_columns["Bin"] == "Small (<1000)"), :]
small_reading_pass_count = small_reading_pass["Bin"].count()

medium_reading_pass = size_reading_pass.loc[(size_columns["Bin"] == "Medium(1000-2000)"), :]
medium_reading_pass_count = medium_reading_pass["Bin"].count()

large_reading_pass = size_reading_pass.loc[(size_columns["Bin"] == "Large (2000-5000)"), :]
large_reading_pass_count = large_reading_pass["Bin"].count()

#Reading Passing percentage of Each Size 

small_reading_per = (small_reading_pass_count/small_count) *100
medium_reading_per = (medium_reading_pass_count/medium_count) *100
large_reading_per = (large_reading_pass_count/large_count) *100


#Both Passing of Each Size

size_both_pass = size_columns.loc[(size_columns["Passed Both?"] == True), :]

small_both_pass = size_both_pass.loc[(size_columns["Bin"] == "Small (<1000)"), :]
small_both_pass_count = small_both_pass["Bin"].count()

medium_both_pass = size_both_pass.loc[(size_columns["Bin"] == "Medium(1000-2000)"), :]
medium_both_pass_count = medium_both_pass["Bin"].count()

large_both_pass = size_both_pass.loc[(size_columns["Bin"] == "Large (2000-5000)"), :]
large_both_pass_count = large_both_pass["Bin"].count()

#Reading Passing percentae  of Each Size 

small_both_per = (small_both_pass_count/small_count) *100
medium_both_per = (medium_both_pass_count/medium_count) *100
large_both_per = (large_both_pass_count/large_count) *100




school_size_summary_df = pd.DataFrame({"School Size": ["Small (<1000)","Medium(1000-2000)","Large (2000-5000)"],
                                       "Average Math Score": [small_math_avg_per, medium_math_avg_per, large_math_avg_per],
                                       "Average Reading Score":[small_reading_avg_per, medium_reading_avg_per, large_reading_avg_per],
                                       "% Passing Math": [small_math_per,medium_math_per,large_math_per],
                                       "% Passing Reading": [small_reading_per,medium_reading_per,large_reading_per],
                                       "%Overall Passing": [small_both_per, medium_both_per, large_both_per]})
                                       
                                       

school_size_summary_df



,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
0,Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
1,Medium(1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
2,Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


In [10]:
#Scores by School Type

#Pull Relevant Data
school_type = school_data_complete[["type","reading_score", "math_score", "Passed Math?", "Passed Reading?","Passed Both?"]]


#Charter General Data 
charter_data = school_type.loc[(school_type["type"] == "Charter"), :]
charter_count = charter_data["type"].count()
charter_math_avg_per = charter_data["math_score"].mean()
charter_reading_avg_per = charter_data["reading_score"].mean()


#Disctrict General Data
district_data = school_type.loc[(school_type["type"] == "District"), :]
district_count = district_data["type"].count()
district_math_avg_per = district_data["math_score"].mean()
district_reading_avg_per = district_data["reading_score"].mean()

#Math Calculations 

type_math_pass = school_type.loc[(school_type["Passed Math?"] == True), :]

district_math_pass = type_math_pass.loc[(school_type["type"] == "District"), :]
district_math_pass_count = district_math_pass["type"].count()

charter_math_pass = type_math_pass.loc[(school_type["type"] == "Charter"), :]
charter_math_pass_count = charter_math_pass["type"].count()


district_math_per = (district_math_pass_count/district_count)*100
charter_math_per = (charter_math_pass_count/charter_count)*100


#Reading Calculations 

type_reading_pass = school_type.loc[(school_type["Passed Reading?"] == True), :]

district_reading_pass = type_reading_pass.loc[(school_type["type"] == "District"), :]
district_reading_pass_count = district_reading_pass["type"].count()

charter_reading_pass = type_reading_pass.loc[(school_type["type"] == "Charter"), :]
charter_reading_pass_count = charter_reading_pass["type"].count()


district_reading_per = (district_reading_pass_count/district_count)*100
charter_reading_per = (charter_reading_pass_count/charter_count)*100


#Overall Calculations 


type_both_pass = school_type.loc[(school_type["Passed Both?"] == True), :]

district_both_pass = type_both_pass.loc[(school_type["type"] == "District"), :]
district_both_pass_count = district_both_pass["type"].count()

charter_both_pass = type_both_pass.loc[(school_type["type"] == "Charter"), :]
charter_both_pass_count = charter_both_pass["type"].count()


district_both_per = (district_both_pass_count/district_count)*100
charter_both_per = (charter_both_pass_count/charter_count)*100


school_type_summary_df = pd.DataFrame({"School Type": ["Charter","District"],
                                       "Average Math Score": [charter_math_avg_per, district_math_avg_per],
                                       "Average Reading Score":[charter_reading_avg_per, district_reading_avg_per],
                                       "% Passing Math": [charter_math_per,district_math_per],
                                       "% Passing Reading": [charter_reading_per,district_reading_per],
                                       "%Overall Passing": [charter_both_per, district_both_per]})
                                       
                                       

school_type_summary_df


,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
0,Charter,83.406183,83.902821,93.701821,96.645891,90.560932
1,District,76.987026,80.962485,66.518387,80.905249,53.695878
